In [10]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup #Pull data out of HTML and XML.
from selenium import webdriver
import time
import re

url_rank_serie = 'https://www.imdb.com/chart/toptv/'
url_imdb_base = 'https://www.imdb.com'
url_serie_ = f'?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=12230b0e-0e00-43ed-9e59-8d5353703cce&pf_rd_r=BFQY8EMES4H7BK35D10R&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=toptv&ref_=chttvtp_tt_'


In [11]:
def get_html_from_link(page_link):
    '''
        Get HTML from web page and parse it.

        :param page_link: link of the webpage we want to scrap
        :type page_link: string
        :return: BeautifulSoup object (HTML parsed)
        :rtype: bs4.BeautifulSoup
    '''


    # TODO Code this function
    response = requests.get(page_link)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [12]:
html_all = get_html_from_link(url_rank_serie)
print(html_all.prettify())

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="app-id=342792525, app-argument=imdb:///?src=mdot" name="apple-itunes-app"/>
  <style>
   body#styleguide-v2 {
                    background: no-repeat fixed center top #000;
                }
  </style>
  <script type="text/javascript">
   var IMDbTimer={starttime: new Date().getTime(),pt:'java'};
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
  </script>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <title>
   IMDb Top 250 TV - IMDb
  </title>
  <script>
   (function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);
  </script>
  <script>
   if (typeof uet == 'function') {
      uet("be", "LoadTitle",

In [13]:
def get_link_to_serie(root_html):

    """
    This function extract the link to acces the series information page.
    
    :param root_html: BeautifulSoup Element that contains all books links.
    :type book_html: bs4.BeautifulSoup.
    :return: list of all serie links in the page.
    :rtype: list(str).
    """
    serie_links = []
    reg = re.compile('/title/+')
    for elem in root_html.find_all('td', {'class':'titleColumn'}):
        for elements in elem.find_all('a', {'href' : reg}):
            attribut = elements["href"]
            serie_links.append(attribut)
    #len(serie_links)
    return(serie_links)

In [14]:
serie_links=get_link_to_serie(html_all)
#serie_links

In [36]:
def get_info_serie(serie_html):
    
    """
    Return series information
    
    :param serie_html: BeautifulSoup element that contains serie infos.
    :type serie_html: bs4.element.Tag.
    :return:
            - serie_rating : rating of the serie.
            - serie_type: the serie can be TV Mini-Series or TV Series.
    :rtype: tuple(string, string)
    """
    
    serie_title = serie_html.find('h1').text.strip()
    
    serie_nb_episode=serie_html.find('span', {'class':'bp_sub_heading'}).text
        
    serie_genre=[]
    reg_type = re.compile('/search/title')
    for div in serie_html.find_all("div", {"class":"see-more inline canwrap"}):
        if div.find("h4", {"class":"inline"}).text == "Genres:":
            for a in div.find_all("a", {"href": reg_type}):
                genre = a.text
                serie_genre.append(genre)

            
    serie_creators=[]
    reg_name = re.compile('/name/nm+')
    for div in serie_html.find_all("div", {"class":"credit_summary_item"}):
        if div.find("h4", {"class":"inline"}).text == "Creator:":
            for b in div.find_all("a", {"href": reg_name}):
                creators = b.text
                serie_creators.append(creators)
        elif div.find("h4", {"class":"inline"}).text == "Creators:":
            for b in div.find_all("a", {"href": reg_name}):
                creators = b.text
                serie_creators.append(creators)
            
        
    nb_season=[]
    reg_season = re.compile('/title/')
    for div in serie_html.find_all("div", {"class":"seasons-and-year-nav"}):
        for c in div.find_all("a", {"href": reg_season}):
                season_not_filtered = c.text
                nb_season.append(season_not_filtered)
                serie_nb_season = ""
                if nb_season != 1:
                        serie_nb_season = nb_season[0]

            
    return(serie_title,serie_nb_episode,serie_genre,serie_creators,serie_nb_season)

In [37]:
#Parcourt de la liste. La liste commence au rang 0. Donc le premier film est de rang 0 mais de classement 1, soit rang n+1.

rank_number = 0
for n in range(len(serie_links)):
    
    rank_number = n + 1
    #Le rank_number s'ajoute en fin d'url.
    url_serie_rank = url_serie_ + str(rank_number)
    
    # Récupère l'url entier de la page info des séries.
    link_serie = url_imdb_base + serie_links[n] + url_serie_rank
    #print(link_serie) 
    
    html = get_html_from_link(link_serie)
    
    #Application de la fonction get_info_serie pour retourner les information de chaque lien.
    info = get_info_serie(html)
    print(info)
    
    

('Planet Earth II', '6 episodes', [' Documentary'], [], '1')
('Planet Earth', '11 episodes', [' Documentary'], [], '1')
('Band of Brothers', '10 episodes', [' Action', ' Drama', ' History', ' War'], [], '1')
('Breaking Bad', '62 episodes', [' Crime', ' Drama', ' Thriller'], ['Vince Gilligan'], '5')
('Chernobyl', '5 episodes', [' Drama', ' History', ' Thriller'], ['Craig Mazin'], '1')
('The Wire', '60 episodes', [' Crime', ' Drama', ' Thriller'], ['David Simon'], '5')
('Blue Planet II', '8 episodes', [' Documentary'], [], '1')
('Our Planet', '8 episodes', [' Documentary'], [], '1')
('Cosmos: A Spacetime Odyssey', '13 episodes', [' Documentary'], [], '1')
('Game of Thrones', '73 episodes', [' Action', ' Adventure', ' Drama', ' Fantasy', ' Romance'], ['David Benioff', 'D.B. Weiss'], '8')
('Cosmos', '13 episodes', [' Documentary'], ['Ann Druyan', 'Carl Sagan', 'Steven Soter'], '1')
('The Last Dance', '10 episodes', [' Documentary', ' Biography', ' History', ' Sport'], [], '1')
('Rick and M

KeyboardInterrupt: 